## API情報を取得





In [1]:
import requests
import json
import pandas as pd

# APIのひな型
api_key = "8C0D4D319B4B7800DC3A0DE26527806B"
user_key = "76561198030849189"
url= "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={}&steamid={}&format=json&include_appinfo=1".format(api_key,user_key)

In [2]:
r = requests.get(url)
# 結果はJSON形式なのでデコードする
data = json.loads(r.text) 


df = pd.DataFrame(data['response']['games'])  

df[["name","playtime_forever"]].sort_values(by = "playtime_forever",ascending=False)

,name,playtime_forever
70,Crusader Kings II,138327
107,Europa Universalis IV,93258
54,Mount & Blade: Warband,37618
100,Kenshi,22819
56,Terraria,17125
68,X3: Albion Prelude,8226
33,Arsenal of Democracy,6148
49,Darkest Hour: A Hearts of Iron Game,5376
69,Age of Empires® III: Complete Collection,4639
171,The Witcher 3: Wild Hunt,3935
